# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Haoyang"
# Set the data and output directories
DATA_ROOT = '/media/techt/DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Multiple Endocrine Neoplasia Type 2'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [ ]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Multiple-Endocrine-Neoplasia-Type-2"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE107375', 'GSE19987', 'GSE2841', 'GSE67047']

Repeat the below steps for all the accession numbers

In [4]:
cohort = accession_num = "GSE19987" # 19987 biased
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/media/techt/DATA/GEO/Multiple-Endocrine-Neoplasia-Type-2/GSE19987/GSE19987_family.soft.gz',
 '/media/techt/DATA/GEO/Multiple-Endocrine-Neoplasia-Type-2/GSE19987/GSE19987-GPL571_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [5]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Germline Mutations in TMEM127 Confer Susceptibility to Pheochromocytoma"
!Series_summary	"Pheochromocytomas, catecholamine-secreting tumors of neural crest origin, are frequently hereditary. However, the molecular basis of the majority of these tumors is unknown. We identified the transmembrane-encoding gene TMEM127 on chromosome 2q11 as a new pheochromocytoma susceptibility gene. In a cohort of 103 samples, we detected truncating germline TMEM127 mutations in approximately 30% of familial tumors and about 3% of sporadic-appearing pheochromocytomas without a known genetic cause. The wild-type allele was consistently deleted in tumor DNA, suggesting a classic mechanism of tumor suppressor gene inactivation. Pheochromocytomas with mutations in TMEM127 are transcriptionally related to tumors bearing NF1 mutations and, similarly, show hyperphosphorylation of mammalian target of rapamycin (mTOR) effector proteins. Accordingly, in vitro gain-of-function and loss-of-function an

In [6]:
clinical_data.head()

,!Sample_geo_accession,GSM499539,GSM499540,GSM499541,GSM499542,GSM499543,GSM499544,GSM499545,GSM499546,GSM499547,...,GSM499579,GSM499580,GSM499581,GSM499582,GSM499583,GSM499584,GSM499585,GSM499586,GSM499587,GSM499588
0,!Sample_characteristics_ch1,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,...,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma,tumor type: pheochromocytoma
1,!Sample_characteristics_ch1,genetic class: NF1,genetic class: VHL,genetic class: SPOR,genetic class: B_SDHB,genetic class: SPOR,genetic class: SPOR,genetic class: SPOR,genetic class: SPOR,genetic class: SPOR,...,genetic class: SPOR,genetic class: SPOR,genetic class: SPOR,genetic class: NF1,genetic class: FP_TM,genetic class: SPOR,genetic class: SPOR,genetic class: SPOR,genetic class: VHL,genetic class: VHL
2,!Sample_characteristics_ch1,tumor location: unknown,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: unknown,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,...,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: adrenal,tumor location: extraadrenal,tumor location: adrenal,tumor location: adrenal


In [7]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tumor type: pheochromocytoma'],
 1: ['genetic class: NF1',
  'genetic class: VHL',
  'genetic class: SPOR',
  'genetic class: B_SDHB',
  'genetic class: MEN2A',
  'genetic class: MEN2B',
  'genetic class: FP_TM'],
 2: ['tumor location: unknown',
  'tumor location: adrenal',
  'tumor location: extraadrenal']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [8]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. Do remember to separate the value by colon ':' first, as the row header should not be included in the returned value. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Multiple Endocrine Neoplasia Type 2\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Multiple Endocrine Neoplasia Type 2\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique va

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [9]:
age_row = gender_row = None
convert_age = convert_gender = None

In [10]:
is_gene_availabe = True
trait_row = 1
#age_row = 1
#gender_row = 2

trait_type = 'binary'

In [11]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [12]:
# Verify and use the functions generated by GPT

def convert_trait(value: str) -> int:

    """Converts a given MEN2 trait value to a binary format.
    Assumes the value is in the format "genetic class: [class]".
    Returns 1 for MEN2A or MEN2B, 0 for other classes, and None for unknown values."""

    try:
        # Separate the value by colon and extract the genetic class part
        genetic_class = value.split(":")[1].strip()
        return 1 if genetic_class in ['MEN2A', 'MEN2B'] else 0
    except IndexError:
        # Handle cases where the genetic class cannot be parsed
        return None


In [13]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM499539,GSM499540,GSM499541,GSM499542,GSM499543,GSM499544,GSM499545,GSM499546,GSM499547,GSM499548,...,GSM499579,GSM499580,GSM499581,GSM499582,GSM499583,GSM499584,GSM499585,GSM499586,GSM499587,GSM499588
Multiple Endocrine Neoplasia Type 2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [14]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM499539,GSM499540,GSM499541,GSM499542,GSM499543,GSM499544,GSM499545,GSM499546,GSM499547,GSM499548,...,GSM499579,GSM499580,GSM499581,GSM499582,GSM499583,GSM499584,GSM499585,GSM499586,GSM499587,GSM499588
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,91.4244,26.1889,77.6878,53.7060,78.0359,175.5990,60.35620,73.2733,166.8090,64.8903,...,420.3310,502.8970,391.8470,782.8080,586.5700,337.0960,432.4340,434.7950,424.2240,626.1600
1053_at,28.3062,38.1084,28.4181,33.0537,34.6472,34.7899,29.56350,17.7059,28.2164,29.7934,...,88.5949,75.9089,83.4956,136.9560,106.4710,71.9423,72.1618,81.3452,94.9625,96.4993
117_at,28.8271,56.4207,32.7921,26.4964,46.3320,297.5880,32.02680,208.1110,32.5678,46.1991,...,104.1940,87.7868,100.5650,142.5530,100.9650,80.7876,318.4840,91.7262,96.7181,103.3320
121_at,89.7293,158.0570,98.9304,110.6610,139.8810,122.4540,141.95400,127.4710,130.7960,117.5070,...,362.9900,426.3530,496.3860,776.8020,610.2740,327.1770,305.9890,402.2010,479.2100,605.8780
1255_g_at,19.4840,29.4318,19.4320,22.4218,23.1822,15.5920,8.18106,25.0809,17.5423,16.0541,...,42.2602,46.8180,31.6632,55.9251,48.0767,37.2250,37.3452,36.9560,40.2922,39.2037


In [15]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1598_g_at',
 '160020_at',
 '1729_at',
 '1773_at',
 '177_at',
 '179_at',
 '1861_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [16]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at', '179_at', '1861_at']\n"


If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [18]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [19]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at', '179_at', '1861_at']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Spec

In [20]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [21]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

176 input query terms found dup hits:	[('ABCC6P1', 2), ('ABCC6P2', 3), ('ABCD1P2', 2), ('ACTBP9', 2), ('AGAP11', 2), ('AGAP7P', 2), ('ANKH
210 input query terms found no hit:	['AK055967', 'ALPPL2', 'APITD1-CORT', 'ATP5G2P1', 'ATP5J2', 'AX746733', 'BMS1P5', 'C15orf38-AP3S2', 
71 input query terms found dup hits:	[('SENP3-EIF4A1', 2), ('SERF2', 2), ('SETP4', 2), ('SHC1P1', 2), ('SIRPAP1', 2), ('SLC7A5P2', 2), ('
86 input query terms found no hit:	['SEPT5', 'SEPT6', 'SMA4', 'SMA5', 'SNORA45A', 'SOX5P', 'SPHAR', 'TCEB1P28', 'TCEB1P3', 'TCR-alpha',
25 input query terms found dup hits:	[('APOBEC3F', 2), ('APOLD1', 2), ('ARF1P1', 2), ('ARHGEF4', 2), ('ARID4B', 2), ('ARMC2-AS1', 2), ('A
155 input query terms found no hit:	['APITD1', 'ARMC4', 'ARNTL', 'ARNTL2', 'ARSE', 'ASNA1', 'ASUN', 'ATHL1', 'ATP5A1', 'ATP5B', 'ATP5C1'
24 input query terms found dup hits:	[('CCPG1', 2), ('CCZ1', 2), ('CDC37P1', 2), ('CDR1', 2), ('CDY1', 2), ('CENPA', 2), ('CFH', 2), ('CK
31 input query terms found no hit:	['

In [22]:
genetic_data

,GSM499539,GSM499540,GSM499541,GSM499542,GSM499543,GSM499544,GSM499545,GSM499546,GSM499547,GSM499548,...,GSM499579,GSM499580,GSM499581,GSM499582,GSM499583,GSM499584,GSM499585,GSM499586,GSM499587,GSM499588
A1CF,71.557100,159.891000,57.015500,120.95900,127.401000,95.989100,118.137000,101.873000,63.52370,161.5950,...,142.776000,104.520000,137.614000,175.06000,165.580000,100.254000,92.139200,102.317000,145.369000,136.1520
A2M,408.900000,202.328000,210.067000,381.63500,491.492000,164.369000,831.625000,323.033000,205.33400,471.6100,...,1152.810000,869.373000,998.160000,786.61300,774.388000,1006.760000,369.229000,408.519000,744.746000,924.6230
A4GALT,12.690400,21.980000,14.371900,28.56820,19.156600,12.430800,5.653090,13.445900,8.82465,29.9103,...,112.240000,93.937300,146.112000,145.40600,130.332000,87.782200,85.761200,98.144000,104.492000,114.6330
A4GNT,37.140700,44.172100,28.034900,40.66690,39.553300,35.653100,27.099000,26.327400,37.09310,41.3090,...,42.834000,38.710000,29.049800,43.55150,41.107700,37.681900,33.042500,41.756200,46.398500,47.7418
AAAS,29.980400,23.644500,30.930100,23.51450,31.946500,30.712800,6.666480,34.113000,50.83830,24.3827,...,104.784000,96.517000,123.071000,147.06500,129.080000,84.828800,92.175300,103.904000,101.345000,135.3390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDB,13.133700,14.893200,4.323000,18.05100,18.067800,13.865300,13.798900,14.533800,11.35270,13.2336,...,25.520200,31.749500,50.974300,49.25600,49.769500,34.056500,33.746200,30.191100,51.225400,54.8993
ZXDC,22.318400,2.990380,9.406920,5.84395,3.589870,11.439900,18.574500,7.238380,13.43450,16.3209,...,110.979000,92.301900,103.830000,112.92100,110.700000,123.282000,80.434700,116.624000,113.119000,96.7548
ZYX,57.948950,41.521600,44.554350,59.32980,45.051800,46.761650,52.722850,38.256300,51.66035,43.5322,...,74.040050,87.582350,111.134800,88.40755,73.397850,69.180850,76.572700,75.683800,76.092850,59.1089
ZZEF1,33.593887,23.616997,39.444033,26.66220,28.433633,30.751333,25.160033,27.164767,36.67980,31.6776,...,70.053667,78.968233,80.725033,124.36110,92.265033,67.288667,81.287033,79.676367,89.155567,104.3315


In [23]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [24]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 50 samples.


In [25]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Multiple Endocrine Neoplasia Type 2', the least common label is '1.0' with 4 occurrences. This represents 8.00% of the dataset.
The distribution of the feature 'Multiple Endocrine Neoplasia Type 2' in this dataset is severely biased.



False

In [26]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [27]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [28]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE19987,True,True,False,False,False,50,


In [29]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [30]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [31]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

In [ ]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 4. Discussion and report

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)